In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from neutorch.dataset.border_mask import create_border_mask

AFF_BORDER_WIDTH = 2
def compute_affinity(label):

    z0, y0, x0 = label.shape

    # add background mask
    masked_label = np.zeros(label.shape, dtype=np.uint64)
    create_border_mask(label, masked_label, AFF_BORDER_WIDTH, 0)

    # along some axis X, affinity is 1 or 0 based on if voxel x === x-1
    affinity = np.zeros((3, z0, y0, x0))
    affinity[2, 1:, :, :] = masked_label[..., 1:, :,
                                            :] == masked_label[..., 0:-1, :, :]  # z channel
    affinity[1, :, 1:, :] = masked_label[..., :, 1:,
                                            :] == masked_label[..., :, 0:-1, :]  # y channel
    affinity[0, :, :, 1:] = masked_label[..., :, :,
                                            1:] == masked_label[..., :, :, 0:-1]  # x channel

    # but back in background labels
    affinity[:, masked_label == 0] = 0

    return affinity


In [ ]:
from neutorch.dataset.utils import from_h5

path = '../../data'
file = 'sample_A'

image = from_h5(f'{path}/{file}.hdf', dataset_path='volumes/raw')
label = from_h5(
f'{path}/{file}.hdf', dataset_path='volumes/labels/neuron_ids')
target = 

In [ ]:
from neutorch.dataset.proofread import ProofreadDataset
dataset = ProofreadDataset(image, label, name=file)

In [19]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from neutorch.model.ViT import ViT

v = ViT(
    patch_size = 4,
    num_classes = 1000,
    dim = 2,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
)

img = torch.randn(1, 3, sz, sz)

preds = v(img) # (1, 1000)
print(preds.shape)

TypeError: __init__() got an unexpected keyword argument 'image_size'

In [27]:
from einops import rearrange

img = torch.randn(1, 3, 128, 256, 256)
img = img.to_sparse()

x = rearrange(img, 'b c (d pz) (h py) (w px) -> b (d h w) (pz py px c)',
                      pz=4, py=4, px=4)

print(x.shape)

RuntimeError: reshape is not implemented for sparse tensors

plt.imshow(img[0])
plt.show()